In [1]:
import sys 
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.features import preprocessing
from src.features import sequences
from src.features import knowledge
from pathlib import Path
from collections import Counter
import pandas as pd

In [2]:
data_path = Path('/home/ralmasri/projects/Thesis/Domain-Guided-Monitoring/data/')
csv_path = data_path / "logs_aggregated_concurrent.csv"
size = 50
# data_df = pd.read_csv(csv_path).head(size) if size > 0  else pd.read_csv(csv_path)
subset_path = data_path / f"{size}_logs_aggregated_concurrent.csv" if size > 0 else csv_path
# if size > 0:
#     data_df.to_csv(subset_path)

In [3]:
huawei_config = preprocessing.HuaweiPreprocessorConfig()
huawei_config.aggregated_log_file = subset_path
sequence_preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
sequence_column_name = sequence_preprocessor.sequence_column_name
sequence_df = sequence_preprocessor.load_data()
sequence_df

Generating DRAIN clusters from log_df: 100%|██████████| 15/15 [00:00<00:00, 1939.83it/s]


,num_logs,num_events,all_events,attributes,coarse_log_cluster_template,fine_log_cluster_template
0,50,50,[[coarse_log_cluster_template#19 nov 2019 17 *...,"[[http_method#get, http_status#200.0, url_clus...",[[coarse_log_cluster_template#19 nov 2019 17 *...,[[fine_log_cluster_template#19 nov 2019 17 * *...


In [4]:
sequence_config = sequences.SequenceConfig()
transformer = sequences.NextSequenceTransformer(sequence_config)
metadata = transformer.collect_metadata(sequence_df=sequence_df, sequence_column_name='all_events')
metadata.x_vocab

{'programname#neutron-server': 0,
 'python_module#neutron.wsgi': 1,
 'coarse_log_cluster_template#get v20 ports tenant id 99c2677b197747c9bd8bc0d8a44a0fb6 device id 7c9e94497d8140d082dbe3e0536f5afd http 11 status 200 len 210 time 00277810': 2,
 'coarse_log_cluster_template#19 nov 2019 17 03 * 0100 post v3 auth tokens http 11 201 6856 * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712': 3,
 'http_method#post': 4,
 'fine_log_cluster_template#19 nov 2019 17 03 * delete v2 images * http 11 204 213 *': 5,
 'http_status#300.0': 6,
 'fine_log_cluster_template#19 nov 2019 17 03 39 get v2 schemas image http 11 200 5562 *': 7,
 'http_status#404.0': 8,
 'programname#keystone-apache-public-access': 9,
 'fine_log_cluster_template#19 nov 2019 17 03 * 0100 get http 11 300 270 * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712': 10,
 'coarse_log_cluster_template#19 nov 2019 17 03 22 0100 get v3 domains name default http 11 200 293 43285 pythonkeystoneclient': 11,
 'coarse_log_cluster

In [5]:
causality_preprocessor = preprocessing.ConcurrentAggregatedLogsCausalityPreprocessor(config=huawei_config)
causality_df = causality_preprocessor.load_data()
causality = knowledge.CausalityKnowledge(config=knowledge.KnowledgeConfig())

Generating causality df from counted causality: 100%|██████████| 66/66 [00:00<00:00, 19744.94it/s]


In [6]:
huawei_df = sequence_preprocessor._load_log_only_data().fillna("")
huawei_df.head(5)

Generating DRAIN clusters from log_df: 100%|██████████| 15/15 [00:00<00:00, 3968.37it/s]


,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,timestamp,url_cluster_template,url_cluster_path
0,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:00:48.274000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,get v20 networks shared true http 11 status 20...,13 get v20 networks shared true http * status *,get v20 networks shared true http 11 status 20...,13 get v20 networks,2019-11-19 17:00:48.274000+01:00,,
1,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:01:50.119000000+01:00,"130.149.249.127 ""GET /v2.0/ports?fields=bindin...",,get v20 ports fields binding 3ahost id fields ...,22 get v20 ports fields binding * id fields bi...,get v20 ports fields binding 3ahost id fields ...,22 get v20 ports,2019-11-19 17:01:50.119000+01:00,,
2,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:01:50.210000000+01:00,"130.149.249.127 ""GET /v2.0/ports?tenant_id=99c...",,get v20 ports tenant id 99c2677b197747c9bd8bc0...,17 get v20 ports tenant id * device id *,get v20 ports tenant id 99c2677b197747c9bd8bc0...,17 get v20 ports,2019-11-19 17:01:50.210000+01:00,,
3,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:01:50.317000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,get v20 networks shared true http 11 status 20...,13 get v20 networks shared true http * status *,get v20 networks shared true http 11 status 20...,13 get v20 networks,2019-11-19 17:01:50.317000+01:00,,
4,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:02:52.398000000+01:00,"130.149.249.127 ""GET /v2.0/networks?tenant_id=...",,get v20 networks tenant id 99c2677b197747c9bd8...,16 get v20 networks tenant id * shared false http,get v20 networks tenant id 99c2677b197747c9bd8...,16 get v20 networks,2019-11-19 17:02:52.398000+01:00,,


In [7]:
relevant_columns = set(
            [
                x
                for x in sequence_preprocessor.relevant_columns
                if not causality_preprocessor.config.log_only_causality or "log" in x
            ]
        )
relevant_columns

{'Hostname',
 'coarse_log_cluster_template',
 'fine_log_cluster_template',
 'http_method',
 'http_status',
 'log_level',
 'programname',
 'python_module',
 'url_cluster_template'}

In [9]:
counted_causality = causality_preprocessor._generate_counted_causality(huawei_df, sequence_preprocessor.relevant_columns)
counted_causality['fine_log_cluster_template#' + huawei_df['fine_log_cluster_template'].iloc[0]]

Generating counted causality for Huawei log data: 100%|██████████| 50/50 [00:00<00:00, 370.79it/s]


['coarse_log_cluster_template#get v20 ports fields binding 3ahost id fields binding 3avif type device id 7c9e94497d8140d082dbe3e0536f5afd http 11 status 200 len 210 time 00377178',
 'fine_log_cluster_template#get v20 ports fields binding 3ahost id fields binding 3avif type device id 7c9e94497d8140d082dbe3e0536f5afd http 11 status 200 len 210 time 00377178',
 'programname#neutron-server',
 'Hostname#wally113',
 'log_level#info',
 'python_module#neutron.wsgi',
 'coarse_log_cluster_template#get v20 networks tenant id 99c2677b197747c9bd8bc0d8a44a0fb6 shared false http 11 status 200 len 213 time *',
 'fine_log_cluster_template#get v20 networks tenant id 99c2677b197747c9bd8bc0d8a44a0fb6 shared false http 11 status 200 len 213 time *',
 'programname#neutron-server',
 'Hostname#wally113',
 'log_level#info',
 'python_module#neutron.wsgi']

In [10]:
causality_records = []
min_causality = 0.01
for from_value, to_values in counted_causality.items():
    total_to_counts = len(to_values)
    to_values_counter = Counter(to_values)
    for to_value, to_count in to_values_counter.items():
        if to_count / total_to_counts > min_causality:
            causality_records.append(
                {
                    "parent_id": from_value,
                    "parent_name": from_value.split("#")[1],
                    "child_id": to_value,
                    "child_name": to_value.split("#")[1],
                },
            )

result = pd.DataFrame.from_records(causality_records).drop_duplicates().reset_index(drop=True)
result

,parent_id,parent_name,child_id,child_name
0,coarse_log_cluster_template#get v20 networks s...,get v20 networks shared true http 11 status 20...,coarse_log_cluster_template#get v20 ports fiel...,get v20 ports fields binding 3ahost id fields ...
1,coarse_log_cluster_template#get v20 networks s...,get v20 networks shared true http 11 status 20...,fine_log_cluster_template#get v20 ports fields...,get v20 ports fields binding 3ahost id fields ...
2,coarse_log_cluster_template#get v20 networks s...,get v20 networks shared true http 11 status 20...,programname#neutron-server,neutron-server
3,coarse_log_cluster_template#get v20 networks s...,get v20 networks shared true http 11 status 20...,Hostname#wally113,wally113
4,coarse_log_cluster_template#get v20 networks s...,get v20 networks shared true http 11 status 20...,log_level#info,info
...,...,...,...,...
675,python_module#glance_store._drivers.rbd,glance_store._drivers.rbd,fine_log_cluster_template#19 nov 2019 17 03 * ...,19 nov 2019 17 03 * delete v2 images * http 11...
676,python_module#glance_store._drivers.rbd,glance_store._drivers.rbd,programname#glance-api,glance-api
677,python_module#glance_store._drivers.rbd,glance_store._drivers.rbd,Hostname#wally113,wally113
678,python_module#glance_store._drivers.rbd,glance_store._drivers.rbd,log_level#info,info
